<a href="https://colab.research.google.com/github/g-aditi/moral-compass/blob/Nihaarika/Moral_Compass_VectorDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install PyPDF2

In [ ]:
import PyPDF2
import pandas as pd
import numpy as np
import os
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
def pdf_to_text(pdf_data_directory, txt_data_directory):
    # Converting pdf version paper to txt file

    if not os.path.exists(txt_data_directory):
        os.makedirs(txt_data_directory)

    for pdf_filename in os.listdir(pdf_data_directory):

        if pdf_filename.endswith(".pdf"):

            pdf_path = os.path.join(pdf_data_directory, pdf_filename)
            #checkpoint
            #print (pdf_filename)
            txt_filename = os.path.splitext(pdf_filename)[0] + ".txt"
            txt_path = os.path.join(txt_data_directory, txt_filename)

            with open(pdf_path, "rb") as pdf_file:
                pdf_reader = PyPDF2.PdfReader(pdf_file)
                text = ""
                for page_num in range(len(pdf_reader.pages)):
                    page = pdf_reader.pages[page_num]
                    text += page.extract_text()

                with open(txt_path, "w", encoding="utf-8") as txt_file:
                    txt_file.write(text)

pdf_data_directory = "/content/MoralCompass_Papers_VDB_pdf"
txt_data_directory = "/content/MoralCompass_Papers_VDB_txt"
pdf_to_text(pdf_data_directory, txt_data_directory)



Procedures-for-HSR-2022.pdf
466535.pdf


In [72]:
# training a document vectorizer model

txt_data_directory = "/content/MoralCompass_Papers_VDB_txt"
txt_file_paths = [os.path.join(txt_data_directory, file) for file in os.listdir(txt_data_directory) if file.endswith('.txt')]
tagged_data = []
for i, txt_file_path in enumerate(txt_file_paths):
    with open(txt_file_path, 'r', encoding='utf-8') as file:
         text = file.read()
         words = word_tokenize(text)
         words = [word.lower() for word in words]
         tagged_data.append(TaggedDocument(words, tags=['doc_' + str(i)]))

model = Doc2Vec(vector_size=20, epochs=300)
model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

document_vectors = [model.dv['doc_' + str(i)] for i in range(len(tagged_data))]
#checkpoint
#print (document_vectors)

[array([-6.571522 ,  0.742157 , -4.849269 , -2.256128 , -3.6762245,
       -2.5847404,  0.7410783, -3.8210804, -2.2546442, -4.8049073,
        7.0060434,  2.0601761, -1.3135546, -2.2816675,  1.9600258,
       -2.7236683,  1.1966723,  2.8950489, -2.803315 , -0.3542715],
      dtype=float32), array([-1.0299525 ,  0.02386251, -1.0336919 ,  1.2217382 ,  0.9841149 ,
       -1.3641622 , -1.9897507 , -0.9675895 , -0.6963451 , -1.0976708 ,
        1.393137  ,  0.4985668 , -0.98605746, -1.0717086 , -0.05225338,
        0.42312264,  0.7098241 , -0.7046193 , -1.7087755 ,  0.2707506 ],
      dtype=float32)]


In [ ]:
!pip install pandas sentence-transformers pandarallel faiss-cpu



  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 7.4 MB/s eta 0:00:00
  Created wheel for pandarallel: filename=pandarallel-1.6.5-py3-none-any.whl size=16675 sha256=32108e5dbe80671f09bc84f45c912fa83a84078b66c2af7d9dd9b6fc2832b078
  Stored in directory: /root/.cache/pip/wheels/50/4f/1e/34e057bb868842209f1623f195b74fd7eda229308a7352d47f
Successfully built pandarallel


In [ ]:
import faiss

In [73]:
vector_dim = len(document_vectors[1])
#checkpoint
#print (vector_dim)
index = faiss.IndexFlatL2(vector_dim)
vectors_np = np.array(document_vectors).astype('float32')

20
